In [2]:
import os
import re
import string
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from ipynb.fs.full.preprocessing import parse_poems

C:\Users\amanda\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\amanda\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\amanda\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\amanda\AppData\Roaming\P

In [3]:
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

# get words as integers and get the word map
all_words = []
words, word_map = parse_poems(text)
for i in range(len(words)):
    for j in range(len(words[i])):
        all_words.append(words[i][j])
words = all_words
        
n_words = len(words)
n_vocab = len(word_map)

In [4]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 7
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_words - seq_length, 1):
    seq_in = words[i:i + seq_length]
    seq_out = words[i + seq_length]
    dataX.append([word for word in seq_in])
    dataY.append(seq_out)
    
n_sequences = len(dataX)

In [5]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [9]:
# define the LSTM model 
# (single layer with 200 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
# define the checkpoint
filepath="weights-improvement-word-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# start from previous best checkpoint
filename = "checkpoints/weights-improvement-word-50-0.7077.hdf5"
model.load_weights(filename)

In [11]:
# train the model
model.fit(X, y, epochs=50, batch_size=8, callbacks=callbacks_list)

Epoch 1/50
17575/17575 [==============================] - 62s 4ms/step - loss: 0.7774 - acc: 0.8263

Epoch 00001: loss improved from inf to 0.77743, saving model to weights-improvement-word-01-0.7774.hdf5
Epoch 2/50
 3072/17575 [====>.........................] - ETA: 51s - loss: 0.4968 - acc: 0.9173

KeyboardInterrupt: 